# Языковые модели

Языковые модели играют важную роль в системах распознавания речи, помогая создавать более грамотные и лексически корректные тексты. В данной работе мы будем изучать нграмные языковые модели, которые позволяют довольно легко оценить вероятность и правдоподобность текста.

В нграмной языковой модели, нграм - это последовательность из n слов в тексте. Например, в предложении "по-моему мы сэкономим уйму времени если я сойду с ума прямо сейчас", биграмами будут "по-моему мы", "мы сэкономим", "сэкономим уйму" итд. Языковые модели оценивают вероятность появления последовательности слов, исходя из статистики появления каждого из нграм в обучающей выборке.

Порядком (order) нграм языковой модели называют максимальную длину нграм, которую учитывает модель. 

Практическая работа разделена на 2 части: 
1. Построение нграмой языковой модели - основная часть, 10 баллов
1. Предсказание с помощью языковой модели - дополнительная часть, 6 балла



Полезные сслыки:
* arpa формат - https://cmusphinx.github.io/wiki/arpaformat/
* обучающие материалы - https://pages.ucsd.edu/~rlevy/teaching/2015winter/lign165/lectures/lecture13/lecture13_ngrams_with_SRILM.pdf
* обучающие материалы.2 - https://cjlise.github.io/machine-learning/N-Gram-Language-Model/

In [1]:
import numpy as np
from collections import defaultdict
from typing import List, Dict, Tuple

# 1. Построение нграмной языковой модели. (10 баллов)


Вероятность текста с помощью нграмной языковой модели можно вычислить по формуле: 
$$ P(w_1, w_2, .., w_n) = {\prod{{P_{i=0}^{n}(w_i| w_{i-order}, .., w_{i-1})}}} $$

В простом виде, при обучении нграмной языковой модели, чтобы рассчитать условную вероятность каждой нграмы, используется формула, основанная на количестве появлений нграмы в обучающей выборке. Формула выглядит следующим образом:
$$ P(w_i| w_{i-order}, .., w_{i-1}) = {{count(w_{i-order}, .., w_{i})} \over {count(w_{i-order},..., w_{i-1})}} $$

Поскольку униграмы не содержат в себе какого-дибо контекста, вероятность униграмы можно посчитать поделив кол-во этой слова на общее количество слов в обучающей выборке. 


In [2]:
# в первую очередь нам понадобится подсчитать статистику по обучающей выборке 
def count_ngrams(train_text: List[str], order=3, bos=True, eos=True) -> Dict[Tuple[str], int]:
    ngrams = defaultdict(int)
    # TODO реализуйте функцию, которая подсчитывает все 1-gram 2-gram ... order-gram ngram'ы в тексте
    
   
    # Проходим по каждому тексту в обучающей выборке
    for sentence in train_text:
        tokens = sentence.split()  # Разбиваем предложение на токены (слова)
        
        # Добавляем специальные токены начала и конца, если это указано
        if bos:
            tokens = ['<s>'] + tokens
        if eos:
            tokens = tokens + ['</s>']
        
        # Генерируем n-граммы для всех значений от 1 до указанного порядка
        for n in range(1, order + 1):
            for i in range(len(tokens) - n + 1):
                ngram = tuple(tokens[i:i + n])  # Создаем n-грамму
                ngrams[ngram] += 1  # Увеличиваем счетчик данной n-граммы
    
    
    # 
    return dict(ngrams)

In [3]:
def test_count_ngrams():
    assert count_ngrams(['привет привет как дела'], order=1, bos=True, eos=True) == {
        ('<s>',): 1, 
        ('привет',): 2, 
        ('как',): 1, 
        ('дела',): 1, 
        ('</s>',): 1
    }
    assert count_ngrams(['привет привет как дела'], order=1, bos=False, eos=True) == {
        ('привет',): 2, 
        ('как',): 1, 
        ('дела',): 1, 
        ('</s>',): 1
    }
    assert count_ngrams(['привет привет как дела'], order=1, bos=False, eos=False) == {
        ('привет',): 2, 
        ('как',): 1, 
        ('дела',): 1
    }
    assert count_ngrams(['привет привет как дела'], order=2, bos=False, eos=False) == {
        ('привет',): 2, 
        ('как',): 1, 
        ('дела',): 1,
        ('привет', 'привет'): 1,
        ('привет', 'как'): 1,
        ('как', 'дела'): 1
    }    
    assert count_ngrams(['привет ' * 6], order=2, bos=False, eos=False) == {
        ('привет',): 6, 
        ('привет', 'привет'): 5
    }
    result = count_ngrams(['практическое сентября',
                           'второе практическое занятие пройдет в офлайне 32 сентября в 12 часов 32 минуты',
                           'в офлайне в 32 12'], order=5)
    assert result[('<s>',)] == 3
    assert result[('32',)] == 3
    assert result[('<s>', 'в', 'офлайне', 'в', '32')] == 1
    assert result[('офлайне', 'в', '32', '12', '</s>')] == 1
    print('Test 1a passed')
    
    
test_count_ngrams()  

Test 1a passed



Простой подход к вычислению вероятностей через количество нграм имеет существенный недостаток. Если в тексте встретится нграмма, которой не было в обучающей выборке, то вероятность всего текста будет равна нулю. 

Чтобы избежать данного недостатка, вводится специальное сглаживание - add-k сглаживание ([Additive, Laplace smoothing](https://en.wikipedia.org/wiki/Additive_smoothing)). Данная техника позволяет учитывать нграмы, не встретившиеся в обучающей выборке, и при этом не делает вероятность текста равной нулю.

Формула сглаживания Лапласа выглядит следующим образом:

$$ P(w_i| w_{i-order}, .., w_{i-1}) = {{count(w_{i-order}, .., w_{i}) + k} \over {count(w_{i-order},..., w_{i-1}) + k*V}} $$

Здесь V - количество слов в словаре, а k - гиперпараметр, который контролирует меру сглаживания. Как правило, значение k выбирается экспериментально, чтобы найти оптимальный баланс между учетом редких нграм и сохранением вероятности для часто встречающихся нграм.


In [102]:
# функция подсчета вероятности через количество со сглаживанием Лапласа
def calculate_ngram_prob(ngram: Tuple[str], counts: Dict[Tuple[str], int], V=None, k=0) -> float:
    # подсчитывет ngram со сглаживанием Лапласа
    

  # Префикс n-граммы (все слова, кроме последнего)
    ngram_count = 0
    prefix_count = 0

    if len(ngram)>1:
      prefix = ngram[:-1]
    else:                 #если нграм всего одно слово, расчет усложняется, считаем все случаи когда 
      prefix = ngram
      prefix_count = 0
      for gram in counts:
        #print(gram)
        find_flag = True
        i=0
        while find_flag and i < len(gram)-1:
          #for i in range(0, len(gram)-1):
          if gram[i] == xx:
            prefix_count += counts.get(gram, 0)
            find_flag  = False
          i+=1      
              
      prefix_count = prefix_count + counts.get(('<s>',), 0)
      
     

    # Подсчет частоты самой n-граммы и ее префикса
    ngram_count = counts.get(ngram, 0)  # Количество вхождений n-граммы
    
    if prefix_count == 0:
      prefix_count = counts.get(prefix, 0)  # Количество вхождений префикса
        
    V = len(set([ngram[-1] for ngram in counts.keys()]))
    #print(V)
    
    # Вычисление вероятности со сглаживанием Лапласа
    prob = (ngram_count + k) / (prefix_count + k * V)

    return prob

In [101]:
def test_calculate_ngram_prob():
    counts = count_ngrams(['практическое сентября',
                           'второе практическое занятие в офлайне 32 сентября в 12 часов 32 минуты',
                           'в офлайне в 32 12'], order=4)
    assert calculate_ngram_prob(('в', 'офлайне'), counts) == 0.5
    assert calculate_ngram_prob(('в', ), counts) == 4/25
    assert calculate_ngram_prob(('в', ), counts, k=0.5) == (4+0.5)/(25+0.5*12)
    assert calculate_ngram_prob(('в', 'офлайне', 'в', '32'), counts) == 1.0
    assert calculate_ngram_prob(('в', 'офлайне'), counts, k=1) == 0.1875
    assert calculate_ngram_prob(('в', 'офлайне'), counts, k=0.5) == 0.25
    assert calculate_ngram_prob(('в', 'онлайне'), counts, k=0) == 0.0
    assert calculate_ngram_prob(('в', 'онлайне'), counts, k=1) == 0.0625
    assert calculate_ngram_prob(('в', 'офлайне'), counts, k=0.5) == 0.25

    print("Test 1.b passed")
    

test_calculate_ngram_prob()  

Test 1.b passed


Основной метрикой язковых моделей является перплексия. 

Перплексия  — безразмерная величина, мера того, насколько хорошо распределение вероятностей предсказывает выборку. Низкий показатель перплексии указывает на то, что распределение вероятности хорошо предсказывает выборку.

$$ ppl = {P(w_1, w_2 ,..., w_N)^{- {1} \over {N}}} $$


In [159]:
# Языковая модель 
import math

class NgramLM:
    def __init__(self, order=3, bos=True, eos=True, k=1, predefined_vocab=None):
        self.order = order
        self.eos = eos
        self.bos = bos
        self.k = k
        self.vocab = predefined_vocab
        self.ngrams_count = None
        
    @property
    def V(self) -> int:
        return len(self.vocab)
    
    def fit(self, train_text: List[str]) -> None:
        # TODO
        # Подсчет vocab и ngrams_count по обучающей выборке
        # Проходим по каждому тексту в обучающей выборке
        ngrams = defaultdict(int)
        vocab = set()
        
        for sentence in train_text:
            tokens = sentence.split()  # Разбиваем предложение на токены (слова)
            
            # Добавляем специальные токены начала и конца, если это указано
            if self.bos:
                tokens = ['<s>'] + tokens
            if self.eos:
                tokens = tokens + ['</s>']

            vocab.update(tokens)
            
            # Генерируем n-граммы для всех значений от 1 до указанного порядка
            for n in range(1, self.order + 1):
                for i in range(len(tokens) - n + 1):
                    ngram = tuple(tokens[i:i + n])  # Создаем n-грамму
                    ngrams[ngram] += 1  # Увеличиваем счетчик данной n-граммы
        
        self.ngrams_count = dict(ngrams)
        if self.vocab is None:
            self.vocab = list(vocab)
    
                
    
    def predict_ngram_log_proba(self, ngram: Tuple[str]) -> float:
        # TODO 
        # считаем логарифм вероятности конкретной нграмы
        prob = calculate_ngram_prob(ngram, counts = self.ngrams_count, V=self.V, k=self.k)
        return math.log(prob)
           
    def predict_log_proba(self, words: List[str]) -> float:
        if self.bos:
            words = ['<s>'] + words
        if self.eos:
            words = words + ['</s>']
        logprob = 0
        # TODO 
        # применяем chain rule, чтобы посчитать логарифм вероятности всей строки
        for i in range(1, len(words) + 1):
            ngram = tuple(words[max(0, i - self.order):i])
            logprob += self.predict_ngram_log_proba(ngram)

        return logprob
        
    def ppl(self, text: List[str]) -> float:
        #TODO 
        # подсчет перплексии
        # Для того, чтобы ваш код был численно стабильным, 
        #    не считайте формулу напрямую, а воспользуйтесь переходом к логарифмам вероятностей
        log_prob_sum = 0
        total_words = 0
        
        for sentence in text:
            words = sentence.split()
            total_words += len(words)
            log_prob_sum += self.predict_log_proba(words)
            print(log_prob_sum,total_words)
    
        perplexity = math.exp(-log_prob_sum / total_words)
        #perplexity = abs(log_prob_sum**-(1 / total_words))
        # 
        return perplexity

In [160]:
def test_lm():
    train_data = ["по-моему мы сэкономим уйму времени если я сойду с ума прямо сейчас",
                  "если я сойду с ума прямо сейчас по-моему мы сэкономим уйму времени",
                  "мы сэкономим уйму времени если я сейчас сойду с ума по-моему"]
    global lm
    lm = NgramLM(order=2)
    lm.fit(train_data)
    assert lm.V == 14
    assert np.isclose(lm.predict_log_proba(['мы']), lm.predict_log_proba(["если"]))
    assert lm.predict_log_proba(["по-моему"]) > lm.predict_log_proba(["если"]) 
    
    
    gt = ((3+1)/(41 + 14) * 1/(3+14))**(-1/2)
    ppl = lm.ppl([''])
    assert  np.isclose(ppl, gt), f"{ppl=} {gt=}"
    
    gt = ((3+1)/(41 + 14) * 1/(3+14) * 1/(14)) ** (-1/3)
    ppl = lm.ppl(['ЧТО'])
    assert  np.isclose(ppl, gt), f"{ppl=} {gt=}"
    
    test_data = ["по-моему если я прямо сейчас сойду с ума мы сэкономим уйму времени"]
    ppl = lm.ppl(test_data)
    assert round(ppl, 2) == 7.33, f"{ppl}"
test_lm()

-4.2801323269925415 0


ZeroDivisionError: float division by zero

# 2. Предсказания с помощью языковой модели (6 балла)

In [161]:
def predict_next_word(lm: NgramLM, prefix: List[str], topk=4):
    # TODO реализуйте функцию, которая предсказывает продолжение фразы. 
    # верните topk наиболее вероятных продолжений фразы prefix 

    candidates = {}
    
    # Проверяем каждое слово из словаря на возможность продолжить префикс
    for word in lm.vocab:
        ngram = tuple(prefix[-(lm.order - 1):] + [word])  # Формируем n-грамму из префикса и текущего слова
        log_prob = lm.predict_ngram_log_proba(ngram)  # Считаем логарифм вероятности для этой n-граммы
        
        candidates[word] = log_prob  # Сохраняем вероятность для каждого слова
    
    # Сортируем кандидатов по вероятности и выбираем top-k
    sorted_candidates = sorted(candidates.items(), key=lambda item: item[1], reverse=True)
    
    topk_words = [word for word, _ in sorted_candidates[:topk]]  # Возвращаем только слова, не их вероятности
    
    return topk_words 
    

Попробуйте обучить ngram языковую модель на нескольких стихотворениях. Не забудьте трансформировать стихотворение в удобный для ngram модели формат (как сделать так, чтобы модель моделировала рифму?). 
Попробуйте сгенерировать продолжение для стихотворения с помощью такой языковой модели. 

In [249]:
#Your code here 

text_1 = ['Я помню чудное мгновенье: Передо мной явилась ты,  Как мимолетное виденье, Как гений чистой красоты. В томленьях грусти безнадежной, \
В тревогах шумной суеты, Звучал мне долго голос нежный И снились милые черты.',
'Шли годы. Бурь порыв мятежный Рассеял прежние мечты, И я забыл твой голос нежный, Твои небесные черты.',
'В глуши, во мраке заточенья Тянулись тихо дни мои Без божества, без вдохновенья, Без слез, без жизни, без любви.',
'Душе настало пробужденье: И вот опять явилась ты, Как мимолетное виденье, Как гений чистой красоты.',
'И сердце бьется в упоенье, И для него воскресли вновь И божество, и вдохновенье, И жизнь, и слезы, и любовь.',


]

lm_2 = NgramLM(order=4)
lm_2.fit(text_1)

import random

total_pred = []
previous_predict = ['']
prefix =  ['гений']
predict = [prefix[0]]
for i in range(0,40):   
    prefix = predict_next_word(lm=lm_2, prefix=prefix, topk=1)
    
    if previous_predict[0] == prefix[0]:                        # если повтор то делаем рандом
        
        prefix = [total_pred[random.randint(0, len(total_pred)-1)]]
        prefix = [predict_next_word(lm=lm_2, prefix=prefix, topk=5)[random.choice([1,2,3,4])]]
        
        previous_predict = predict
        predict = [prefix[0]]
    
    
    if (prefix[0][0].isupper() or prefix[0] == '</s>') and len(predict)>1 : # новая строка для стихотворения
        print(predict)
        total_pred =  total_pred + predict
        previous_predict = predict
        predict = []
    predict.append(prefix[0])
#print(predict)

['гений', 'чистой', 'красоты.']
['В', 'тревогах', 'шумной', 'суеты,']
['Звучал', 'мне', 'долго', 'голос', 'нежный']
['И', 'сердце', 'бьется', 'в', 'упоенье,']
['красоты.', 'красоты.']
['В', 'тревогах', 'шумной', 'суеты,']
['Звучал', 'мне', 'долго', 'голос', 'нежный']
['И', 'сердце', 'бьется', 'в', 'упоенье,']
